In [7]:
from tardis import run_tardis
import tardis
import plotly.graph_objects as go
from collections import defaultdict

In [44]:
marker_colors = ["#958aff","#ff8b85","#5cff74"]
marker_line_colors = ['#27006b', '#800000', '#00801c']
marker_colors = ['#636EFA', '#EF553B', '#00CC96']

luminosities = ["Emitted", "Absorbed", "Requested"]
X = list(range(1,21))


fig = go.FigureWidget().set_subplots(
        3,1, 
        shared_xaxes=True, 
        vertical_spacing = 0.1,
        row_heights=[0.15, 0.7, 0.15]
    )

for luminosity, marker_color, marker_line_color in zip(luminosities, marker_colors, marker_line_colors):
    fig.add_scatter(
        name = luminosity + "<br>Luminosity", 
        mode='lines+markers', 
        row = 2, 
        col = 1, 
        marker_color=marker_color, 
        marker_line_color=marker_line_color,
        legendgroup = luminosity, 
        marker_line_width=1.5, 
        opacity=0.6
    )
fig.add_scatter(
    name = "Residual<br>Luminosity",
    row = 3, 
    col = 1, 
    hovertext="text",
    marker_color='rgb(158,202,225)', 
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5, 
    mode='lines+markers',
    opacity=0.7

)
fig.add_scatter(
    name = "Next Inner Boundary Temperature",
    row = 1, 
    col = 1, 
    hovertext="text",
    marker_color='rgb(158,202,225)', 
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5, 
    mode='lines+markers',
    opacity=0.7

)


fig = fig.update_layout(
    xaxis = dict(
        range = [0, 21],
        dtick = 2,
        
    ),
    xaxis2 = dict(
        matches = "x",
        range = [0, 21],
        dtick = 2,
    ),
    xaxis3 = dict(
        matches = "x",
        title = r"$\mbox{Iteration Number}$", 
        dtick = 2,
    ),
    
    yaxis = dict(
        title = r"$\mbox{T}_{inner}$",
        automargin = True,
        side = "top",
        exponentformat = "e",
    ),
    yaxis2 = dict(
        exponentformat = "e",
        title = r"$\mbox{Luminosity}~(erg~sec^{-1})$",
        title_font_size = 13,
        automargin = True
    ),
    yaxis3 = dict(
        exponentformat = "e",
        title = r"$~~\mbox{Residual}\\\mbox{Luminosity(%)}$", 
        title_font_size = 12,
        automargin = True
    ),
    
    legend_tracegroupgap = 0,
    hidesources = True,
    height = 600,
    hoverlabel_align = 'right',
    legend_title_text = "Luminosity",
)

   

In [45]:
fig

FigureWidget({
    'data': [{'legendgroup': 'Emitted',
              'marker': {'color': '#636EFA', 'line': {'…

In [46]:
traces = defaultdict(list)

In [47]:
def update_traces(sim):

    global traces

    emitted_luminosity = sim.runner.calculate_emitted_luminosity(
        sim.luminosity_nu_start, sim.luminosity_nu_end
    )
    reabsorbed_luminosity = sim.runner.calculate_reabsorbed_luminosity(
        sim.luminosity_nu_start, sim.luminosity_nu_end
    )

    traces["Emitted"].append(emitted_luminosity.value)
    traces["Absorbed"].append(reabsorbed_luminosity.value)
    traces["Requested"].append(sim.luminosity_requested.value)
    traces["t_inner"].append(sim.model.t_inner.value)


    with fig.batch_update():
        for index, luminosity in zip(range(3), luminosities):
            fig.data[index].x = X
            fig.data[index].y = traces[luminosity]
            fig.data[index].hovertemplate = '<b>%{y:.2g}</b>'+'<br>at X = %{x}<br>'
    
        y = [((emitted-requested)*100)/requested for emitted, requested in zip(traces["Emitted"],traces["Requested"])]
        fig.data[-2].x = X
        fig.data[-2].y = y
        fig.data[-2].hovertemplate='Residual Luminosity: %{y:.2f}% at X = %{x:,.0f}<extra></extra>'
        
        


        fig.data[-1].x = X
        fig.data[-1].y = traces["t_inner"]
        fig.data[-1].hovertemplate='Next Inner Body Temperature: %{y:.2f} at X = %{x:,.0f}<extra></extra>'
        
        

In [48]:
sim.model.t_inner.value

11136.097470940847

In [49]:
sim = run_tardis('tardis_example.yml', 
                 simulation_callbacks=[[update_traces]])

[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/atharva/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 7.94275e+42 erg / s Luminosity absorbed = 2.65805e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad    n

[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/atharva/.conda/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning:

divide by zero encountered in double_scalars

 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.33305e+43 erg / s Luminosity absorbed = 4.33011e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10569.747403  11547.678162  0.442290  0.500345
	5      11099.194259  11669.739508  0.153253  0.198338
	10     10809.397890  11493.569590  0.097256  0.121915
	15     10454.841299  11083.769586  0.073809  0.0928

[tardis.simulation.base][INFO   ]  Starting iteration 15/20 (base.py:323)
[py.warnings         ][WARNING]  /home/atharva/.conda/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning:

divide by zero encountered in double_scalars

 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.75331e+42 erg / s Luminosity absorbed = 2.81319e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11437.188130  10637.774893  0.496749  0.452668
	5      11571.106856  11002.647927  0.196714  0.166197
	10     11378.137918  10832.042108  0.120954  0.101284
	15     10957.211814  10350.602742  0.092896  0.079468

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10152.603 K -- next t_inner 11168.

In [291]:
fig.show(renderer = "notebook_connected")

In [382]:
fig.show(renderer = "notebook_connected")